In [ ]:
import os
import pandas as pd
from bs4 import BeautifulSoup
import chardet

In [ ]:
SCORE_DIR = "data/scores"

In [ ]:
box_scores = os.listdir(SCORE_DIR)

In [ ]:
box_scores = [os.path.join(SCORE_DIR, f) for f in box_scores if f.endswith(".html")]

In [ ]:
#cleaning the table and removing some columns/rows
def parse_html(box_score):
    with open(box_score, 'rb') as f:
        result = chardet.detect(f.read())
    detected_charset = result['encoding']
    with open(box_score, encoding=detected_charset) as f:
        html = f.read()
    soup = BeautifulSoup(html)
    [s.decompose() for s in soup.select("tr.over_header")]
    [s.decompose() for s in soup.select("tr.thead")]
    return soup

def read_line_score(soup):
    line_score = pd.read_html(str(soup), attrs={"id": "line_score"})[0]
    home_line_score = line_score.iloc[-1:]
    cols = list(home_line_score.columns)
    cols[0] = "team"
    cols[-1] = "total"
    home_line_score.columns = cols
    home_line_score = home_line_score[["team", "total"]]

    away_line_score = line_score.iloc[:1]
    cols = list(away_line_score.columns)
    cols[0] = "team_opp"
    cols[-1] = "total_opp"
    away_line_score.columns = cols
    away_line_score = away_line_score[["team_opp", "total_opp"]]

    home_line_score =home_line_score.reset_index(drop=True)
    away_line_score = away_line_score.reset_index(drop=True)

    new_line_score = pd.concat([home_line_score, away_line_score], axis=1)
    return new_line_score

def read_stats(soup, team, stat):
    df = pd.read_html(str(soup), attrs={"id": f"box-{team}-game-{stat}"}, index_col=0)[0]
    df = df.apply(pd.to_numeric, errors="coerce")
    return df

def read_season_info(soup):
    #selecting first in list
    nav = soup.select("#bottom_nav_container")[0]
    hrefs = [a["href"] for a in nav.find_all("a")]
    season = os.path.basename(hrefs[1]).split("_")[0]
    return season

In [ ]:
base_cols = None
games = []
for box_score in box_scores:
    soup = parse_html(box_score)
    line_score = read_line_score(soup)
    teams = list([line_score["team"], line_score["team_opp"]])
    summaries = pd.Series([])
    next_team = False
    for team in teams:
        #get basic and advanced states
        basic = read_stats(soup, team[0], "basic")
        advanced = read_stats(soup, team[0], "advanced")

        #gathering the total of both basic and advance stats and making it one row
        totals = pd.concat([basic.iloc[-1,:], advanced.iloc[-1,:]])
        totals.index = totals.index.str.lower()

        #gathering the max and making it a row
        maxes = pd.concat([basic.iloc[:-1,:].max(), advanced.iloc[:-1,:].max()])
        maxes.index = maxes.index.str.lower() + "_max"

        # concat both max and totals together
        summary = pd.concat([totals, maxes])
        #cleaning cols that we want and removing duplicates
        if base_cols is None:
            base_cols = list(summary.index.drop_duplicates(keep="first"))
            base_cols = [b for b in base_cols if "bpm" not in b]

        summary = summary[base_cols]
        #change title of col for away team
        if next_team:
            summary = summary.add_suffix(f'_opp')
        
        summaries = pd.concat([summaries] + [summary], axis=0)
        next_team = True

    summaries = summaries.to_frame()
    summaries = summaries.transpose()
    game = pd.concat([summaries, line_score], axis=1)
    game["home"] = 1
    game["home_opp"] = 0
    #adding season to help provide information to training model
    game["season"] = read_season_info(soup)

    game["date"] = os.path.basename(box_score)[:8]
    game["date"] = pd.to_datetime(game["date"], format="%Y%m%d")

    game["won"] = game["total"] > game["total_opp"]


    games.append(game)
    
    if len(games) % 100 == 0:
        print(f"{len(games)} / {len(box_scores)}")

In [ ]:
games_df = pd.concat(games, ignore_index=True)

In [ ]:
games_df.to_csv("nba_games_v2.csv")